In [1]:
import numpy as np
import scipy.linalg as la
import copy
from functools import reduce
from pyscf import gto, scf, lo, dft, lib

from pyscf.tools.dump_mat import dump_rec
from sys import stdout
    
mol = gto.Mole()
mol.verbose = 0
mol.atom = '''
He  0.0 0.0  2.0
He  0.0 0.0  4.0
He 0.0 0.0 -2.0
H  0.0 0.0 -2.929352
'''
mol.basis = 'sto-3g'
mol.spin   = 0
mol.charge = 1
mol.build()

mf           = scf.RHF(mol)
mf.conv_tol  = 1e-9
mf.verbose   = 0
mf.max_cycle = 100
mf.kernel()

mf.mulliken_pop()

 ** Mulliken pop  **
pop of  0 He 1s        2.00000
pop of  1 He 1s        2.00000
pop of  2 He 1s        1.77240
pop of  3 H 1s        0.22760
 ** Mulliken atomic charges  **
charge of  0He =      0.00000
charge of  1He =      0.00000
charge of  2He =      0.22760
charge of  3H =      0.77240


(array([2.       , 2.       , 1.7723957, 0.2276043]),
 array([7.84967868e-10, 2.04125605e-12, 2.27604303e-01, 7.72395696e-01]))

In [2]:
w1 = mf.get_ovlp()
dump_rec(stdout, w1[0:2,0:2])
w1[0:2,0:2] = 0.0

        #0        #1       
0       1.00000   0.03202
1       0.03202   1.00000


In [3]:
dm1 = mf.make_rdm1()
dump_rec(stdout, dm1)

        #0        #1        #2        #3       
0       2.00205  -0.06411  -0.00002  -0.00005
1      -0.06411   2.00205  -0.00000   0.00000
2      -0.00002  -0.00000   1.62037   0.34996
3      -0.00005   0.00000   0.34996   0.07558


In [4]:
np.einsum("ij,ji->", w1, dm1)

1.9999999995164532

In [5]:
mol.natm

4

In [6]:
def anydup(thelist):
  seen = set()
  for x in thelist:
    if x in seen: return True
    seen.add(x)
  return False

frgm_list        = [[2,3],[3,6,6,6]]
atm_list_tmp     = []
is_repeated_atom = False
for atm_list in frgm_list:
    for atm_idx in frgm_list:
        is_repeated_atom = atm_idx in atm_list_tmp
        atm_list_tmp.append(atm_idx)
is_repeated_atom

True